# Model Selection Checklist
*(Performance & Robustness focused)*

본 문서는 **추론 모델 선정**을 위해 사용되는 공식 기준 문서이다.  
모든 항목은 주관적 해석이 아닌 **측정 가능한 팩트**에 기반한다.

---

## 1. 모델 선정 체크리스트 (Performance & Robustness)

### A. 비교 공정성 (최소 전제 조건)
- [ ] Train / Test 분할이 고정되어 있으며, 모든 모델 및 실험이 동일 분할을 사용한다
- [ ] Feature-set 정의(e0 / e1 / e2 / e2.1 / e3 / e3.1)는 명확히 고정되어 있고, 실험 간 변경되는 요소는 feature 포함 여부뿐이다
- [ ] 평가지표 산출 방식(ROC-AUC, PR-AUC, Recall 등)이 모든 실험에서 동일하다

---

### C. Robustness (Feature Ablation Stress Test)

#### Leakage / 상태성 의존성
- [ ] e0 → e1에서 성능 감소폭(ΔROC-AUC, ΔPR-AUC, ΔRecall)을 계산했다
- [ ] 성능 감소가 급격하지 않다 (급락 기준은 별도 정의)

#### 결제 정보 의존성
- [ ] e0 → e2(transactions off)에서 성능 감소폭을 계산했다
- [ ] Logs-only 조건에서도 모델이 유의미한 예측력을 유지한다

#### 최신 정보 의존성
- [ ] e0 → e3(w30 off)에서 성능 감소폭을 계산했다
- [ ] 최신 30일 집계 제거 시 성능이 붕괴하지 않는다

#### 최악 조건 검증
- [ ] e3.1 조건에서의 성능을 기록했다
- [ ] e3.1 성능이 베이스라인 모델보다 유의미하게 높다

---

### D. 모델 선정 규칙
- [ ] 최종 선정 지표 1~2개를 사전에 고정했다 (예: PR-AUC, Recall)
- [ ] Robustness 결과(e3.1 성능 또는 감소폭)를 선정 판단에 반영한다
- [ ] 성능 최고 모델이 아닌 경우에도, Robustness가 현저히 우수하면 우선 후보로 고려한다

---

## 2. 모델 탈락 기준표 (Fail-Fast Rules)

아래 조건 중 **하나라도 충족되면 해당 모델은 추론 모델 후보에서 탈락**한다.

| 구분 | 탈락 조건 (팩트 기준) | 의미 |
|---|---|---|
| 누수/상태성 의존 | e1에서 성능이 급락하고, 그 원인이 특정 상태성 컬럼 의존으로 설명됨 | Leakage 가능성 높음 |
| 결제 의존 | e2에서 성능이 운영 불가 수준으로 붕괴 | 결제 없는 사용자 적용 불가 |
| 최신 정보 의존 | e3에서 성능이 급락 | 최신 집계 누락 시 취약 |
| 최악 조건 실패 | e3.1에서 베이스라인 모델보다 유의미하게 못함 | 모델로서 가치 낮음 |
| 일반화 실패 | Validation 대비 Test 성능이 반복적으로 현저히 낮음 | 과적합 |
| 불안정성 | 동일 조건 반복 학습 시 지표 변동 폭이 큼 | 예측 신뢰성 낮음 |
| KPI 미달 | Test 기준 목표 KPI(예: Recall, PR-AUC) 미달 | 비즈니스 요구 불충족 |

※ “급락”, “운영 불가 수준”, “유의미”는 반드시 **숫자 기준으로 정의**해야 한다.  
(예: ΔPR-AUC ≥ 0.15, Recall < 0.6 등)

---

## 3. 문서 사용 원칙

- 본 문서는 **모델을 선택하기 위한 기준**이지, 성능을 미화하기 위한 문서가 아니다
- 성능이 높더라도 탈락 조건을 만족하면 즉시 제외한다
- 최종 선정 모델은 “가장 잘 맞춘 모델”이 아니라 **가장 안정적으로 유지되는 모델**이다

---

## 4. 한 줄 결론

> Feature Ablation 실험을 통과하지 못하는 모델은,  
> 운영 환경에서 신뢰 가능한 추론 모델이 될 수 없다.